In [ ]:
# transformers 라이브러리에서 BERT 토크나이저를 임포트합니다.
from transformers import BertTokenizer

# 다국어 지원 BERT 모델을 사용하여 토크나이저 객체를 생성합니다.
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
import pandas as pd

# pandas를 사용하여 Excel 파일을 읽고 데이터프레임 'df'에 저장합니다. 'openpyxl' 엔진을 사용합니다.
df = pd.read_excel('/content/drive/MyDrive/Project/data.xlsx', engine='openpyxl')

In [ ]:
# 'SENTENCE' 컬럼에 있는 문장들을 토크나이징하고 인코딩합니다.
# 여기서 각 문장에 대한 토크나이징 작업을 수행하고, 텐서 형식으로 반환하며,
# 'max_length'로 최대 길이를 지정하고, 'truncation'과 'padding'을 적용합니다.
encoded_sentences = [tokenizer(sentence, return_tensors='pt', padding='max_length', truncation=True, max_length=128) for sentence in df['sentence']]


In [ ]:
# label에 idx 부여
excel_file = pd.read_excel("/content/drive/MyDrive/Project/A 음식점(15,726).xlsx", engine="openpyxl")

# 중복되지 않는 레이블을 추출하고 인덱스 매핑 딕셔너리 생성
unique_labels = list(set(excel_file["개체명"].str.split(",").str[0].str.strip()))
label2idx = {label: idx for idx, label in enumerate(unique_labels)}

In [ ]:
df

,sentence,label_idx
0,후기 남길게요 오븐 스파게티 서비스로 주세요,0
1,혹시 코스요리에서 파스타만 포장 가능할까요?,0
2,잔치국수 하나 주시겠어요?,22
3,감사합니다,2302
4,볶아 놓은 걸 드리는 게 아니고 바로바로 볶기 때문에 짬뽕 종류는 똑같아요,860
...,...,...
15721,직접 오시는 게 제일 낫고 그다음에는 배달 앱으로 시키는 게 좋아요,651
15722,일 인분이 양이 많은 건 아니죠?,2056
15723,네 면 전문점이에요,1819
15724,준비되는데 10분정도 소요됩니다,389


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터셋 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

# 데이터셋을 준비하기 위해 다시 토크나이저를 초기화합니다.
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# Excel 파일을 읽고 데이터프레임 'df'에 저장합니다.
df = pd.read_excel('/content/drive/MyDrive/Project/data.xlsx')

# 토크나이징과 인코딩을 수행합니다. 이때, 트렁케이션과 패딩을 적용하고 텐서 형식으로 반환합니다.
encoded_data = tokenizer(df['sentence'].tolist(), truncation=True, padding=True, return_tensors='pt')

# 레이블을 인덱싱합니다. 여기서는 'label_idx'라는 컬럼명이 실제 레이블을 포함하고 있다고 가정합니다.
# unique_labels 리스트를 생성하고 이를 이용해 레이블과 인덱스를 매핑하는 딕셔너리를 만듭니다.
unique_labels = list(set(df['label_idx'].tolist()))
label2idx = {label: i for i, label in enumerate(unique_labels)}
# 데이터프레임에서 레이블을 추출하고 이에 해당하는 인덱스를 텐서로 생성합니다.
labels = torch.tensor([label2idx[label] for label in df['label_idx']])

# 인코딩된 데이터와 레이블을 가지고 텍스트 분류 데이터셋 인스턴스를 생성합니다.
dataset = TextClassificationDataset(encoded_data, labels)

# 데이터셋 분할 8:2로
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 데이터 로더 생성
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 모델 로딩
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(unique_labels))
model.to(device)  # 모델을 GPU로 이동, CUDA 사용 가능한 경우

# 옵티마이저와 스케줄러를 설정합니다. 옵티마이저로는 AdamW를 사용하고, 학습률(learning rate)을 설정합니다.
# 스케줄러는 학습을 진행하면서 학습률을 조정하기 위해 사용합니다.
epochs = 5
optimizer = AdamW(model.parameters(), lr=0.001)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import precision_score, f1_score

# 학습 루프
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print('-' * 10)
 # 모델을 학습 모드로 설정합니다.
    model.train()
     # 학습 데이터 로더에서 배치를 순회하면서 학습을 진행합니다.
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}    # 배치 데이터를 적절한 계산 장치로 이동시킵니다.
        outputs = model(**batch)  # 모델에 배치 데이터를 전달하고, 로스를 계산합니다.
        loss = outputs.loss
        loss.backward() # 역전파를 수행합니다.
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad() # 다음 배치를 위해 옵티마이저의 그라디언트를 초기화합니다.

    # 검증 부분
    model.eval()
    total_loss, total_accuracy = 0, 0
    total_precision, total_f1_score = 0, 0
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == batch['labels']).float().mean()
        total_accuracy += accuracy.item()
        total_loss += outputs.loss.item()

        actual_labels = batch['labels'].cpu().numpy()
        predicted_labels = predictions.cpu().numpy()
        precision = precision_score(actual_labels, predicted_labels, average='weighted')
        f1 = f1_score(actual_labels, predicted_labels, average='weighted')
        total_precision += precision
        total_f1_score += f1

    avg_loss = total_loss / len(val_loader)
    avg_accuracy = total_accuracy / len(val_loader)
    avg_precision = total_precision / len(val_loader)
    avg_f1_score = total_f1_score / len(val_loader)
    print(f'Validation loss: {avg_loss}, accuracy: {avg_accuracy}')
    print(f'Validation precision: {avg_precision}, F1-score: {avg_f1_score}')


# 모델 저장
model.save_pretrained('my_model_directory')

In [ ]:
# 로딩 및 인퍼런스 예시
model = BertForSequenceClassification.from_pretrained('my_model_directory')
model.to(device)  # 모델을 적절한 계산 장치로 이동
model.eval()  # 모델을 평가 모드로 설정

test_sentence = "잔치국수 하나 주시겠어요?"
encoded_test = tokenizer(test_sentence, return_tensors='pt')  # 문장 토크나이징

with torch.no_grad():  # 자동 미분 비활성화
    outputs = model(**encoded_test.to(device))  # 모델에 입력 데이터를 전달하여 예측 수행
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)  # 가장 높은 로짓 점수를 가진 클래스를 예측으로 선택

# 예측된 클래스 인덱스를 실제 레이블로 변환
predicted_label = [label for label, index in label2idx.items()]
predicted_class_index = predictions.item()
predicted_class_label = predicted_label[predicted_class_index]

# 실제 레이블 가져오기
actual_class_index = labels[2].item()  # 첫 번째 예제의 실제 레이블
actual_class_label = predicted_label[actual_class_index]

# 예측 결과 출력
print(f"Input sentence: {test_sentence}")
print(f"Actual class label: {actual_class_label}")
print(f"Predicted class label: {predicted_class_label}")

# 실제 레이블과 예측 레이블 비교
if actual_class_index == predicted_class_index:
    print("Prediction is correct!")
else:
    print("Prediction is incorrect.")


Input sentence: 잔치국수 하나 주시겠어요?
Actual class label: 22
Predicted class label: 0
Prediction is incorrect.
